In [1]:
#Create directory structure
import numpy as np
import pandas as pd
import os
import tables
from sklearn.datasets import load_files       
import keras
from keras.utils import np_utils
import numpy as np
from glob import glob
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image                  
from tqdm import tqdm
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

trainingdata_directory = '/home/ubuntu/src/capstone/datafiles/trainingdata/'
trainingdata_spectrograms = '/home/ubuntu/src/capstone/datafiles/trainingdata/training_spectrograms_complete'
os.chdir(trainingdata_directory)
data = pd.read_csv('trainingData.csv')
print(data.head())

languages = np.unique(data['Language'])
print('Total languages: ', len(languages))

print('Keras version: ', keras.__version__)

Using TensorFlow backend.


   Sample Filename     Language
0  000kouqjfnk.mp3    Hmong Daw
1  000w3fewuqj.mp3    Tektiteko
2  000ylhu4sxl.mp3       Teribe
3  0014x3zvjrl.mp3      Chipaya
4  001xjmtk2wx.mp3  KalmykOirat
Total languages:  176
Keras version:  2.1.4


In [ ]:
#Create train and valid data sets, arrange input spectrograms to suit Keras image generator.
#1. Create directories train and valid.
#2. Create 176 directories with language labels inside train and valid directories.

#Code is commented out as this step is required only once.

#Create directories with 176 language names in train and valid directories.
'''
if os.path.exists(trainingdata_spectrograms):
    os.chdir(trainingdata_spectrograms)
    if not os.path.exists('valid'):
        os.makedirs('valid')
    if os.path.exists('valid'):
        os.chdir('valid')
        for language in languages:
            if not os.path.isdir(trainingdata_spectrograms + '/' + 'valid' + '/' + language):
                os.makedirs(trainingdata_spectrograms + '/' + 'valid' + '/' + language)
    os.chdir(trainingdata_spectrograms)
    if not os.path.exists('train'):
        os.makedirs('train')
    if os.path.exists('train'):
        os.chdir('train')
        for language in languages:
            if not os.path.isdir(trainingdata_spectrograms + '/' + 'train' + '/' + language):
                os.makedirs(trainingdata_spectrograms + '/' + 'train' + '/' + language)
'''

In [ ]:
#trainingData.csv is read in the previous step which matches training files to correct language label.
#    Move roughly 20% images for validation remaining images in train directory.
'''
#transfer train and valid data files.
index = 0
if os.path.exists(trainingdata_spectrograms):
    os.chdir(trainingdata_spectrograms)
    
    for language in languages:
        os.chdir(trainingdata_spectrograms)
        index = 0
        if os.path.isdir(language): 
            os.chdir(language)
            for file in os.listdir('.'):
                if os.path.isfile(file):
                    #print(index)
                    if index < 75:
                        destination_path = trainingdata_spectrograms + '/' +  'valid' + '/' + language + '/'  + file
                    else:
                        destination_path = trainingdata_spectrograms + '/' +  'train' + '/' + language + '/'  + file
                    source_path = file
                    os.rename(source_path, destination_path)
                    index += 1

'''

In [2]:
#Taken from Udacity dog project
#Used to fit mode without using Keras image generator. 
#Not useful when using  image generator as Keras image generator flow_from_directory method reads input images automatically.

os.chdir(trainingdata_directory)

def load_dataset(path):
    data = load_files(path)
    language_files = np.array(data['filenames'])
    language_targets = np_utils.to_categorical(np.array(data['target']), 176)
    return language_files, language_targets

# load train, test, and validation datasets
train_files, train_targets = load_dataset(trainingdata_spectrograms + '/' + 'train')
valid_files, valid_targets = load_dataset(trainingdata_spectrograms + '/' + 'valid')

# print statistics about the dataset
print('There are %d total language categories.' % len(languages))
print('There are %s total train spectrgrams.\n' % len(train_files))
print('There are %s total valid spectrgrams.\n' % len(valid_files))


There are 176 total language categories.
There are 52625 total train spectrgrams.

There are 13551 total valid spectrgrams.



In [3]:
#From Udacity dog project. Used to create 4D tensors for Keras.
#This step is not required when using Keras image generator.

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(200, 100))
    # convert PIL.Image.Image type to 3D tensor with shape (200, 100, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 200, 100, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)


In [ ]:

# pre-process the data for Keras
train_tensors1 = paths_to_tensor(train_files[:len(train_files)//2]).astype('float32')/255
train_tensors2 = paths_to_tensor(train_files[len(train_files)//2:]).astype('float32')/255
#valid_tensors = paths_to_tensor(valid_files).astype('float32')/255
#test_tensors = paths_to_tensor(test_files).astype('float32')/255

In [ ]:
#import pickles #Causes memory overflow error (max 4GB allowed, use hdf5)
# Store the 4D tensors on disk. This is doen to avoid calculating 4D tensors each time this jupyter notebook is run
#which is time consuming.
#This step is not required when using Keras image generator.

if not os.path.isfile('train_tensors1.h5'):
    h5file = tables.open_file('train_tensors1.h5', mode='w', title="Test Array")
    root = h5file.root
    h5file.create_array(root, "train_tensors1", train_tensors1)
    h5file.close()

if not os.path.isfile('train_tensors2.h5'):
    h5file = tables.open_file('train_tensors2.h5', mode='w', title="Test Array")
    root = h5file.root
    h5file.create_array(root, "train_tensors2", train_tensors2)
    h5file.close()


In [ ]:
#Read pre-computed tesnors, saves time to generate again.
#This step is not required when using Keras image generator.

h5file = tables.open_file('train_tensors1.h5', mode='r')
root = h5file.root
train_tensors_1 = np.array(np.zeros(66176//2))
for array in h5file.walk_nodes(root , 'Array'):
    train_tensors_1 = np.array(array)
    print('array.shape: ', train_tensors_1.shape)
h5file.close()

train_tensors_2 = np.array(np.zeros(66176//2))
h5file = tables.open_file('train_tensors2.h5', mode='r')
root = h5file.root
for array in h5file.walk_nodes(root , 'Array'):
    train_tensors_2 = np.array(array)
    print('array.shape: ', train_tensors_2.shape)
h5file.close()

In [3]:
#Define CNN model
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD

model = Sequential()
### TODO: Define your architecture.

# 1 Conv Relu BPool 
model.add(Conv2D(filters=16, kernel_size=(3,3), padding='same', activation='relu', 
                        input_shape=(200, 100, 3)))
model.add(MaxPooling2D(pool_size=2, strides=(2,2)))
model.add(BatchNormalization())

# 2 Conv Relu BPool 
model.add(Conv2D(filters=32, kernel_size=(3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2, strides=(2,2)))
model.add(BatchNormalization())

# 3 Conv Relu BPool 
model.add(Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2, strides=(2,2)))
model.add(BatchNormalization())

# 4 Conv Relu BPool 
model.add(Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2, strides=(2,2)))
model.add(BatchNormalization())

# 5 Conv Relu BPool 
model.add(Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2, strides=(2,2)))
model.add(BatchNormalization())

# 6 Conv Relu BPool 
model.add(Conv2D(filters=256, kernel_size=(3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2, strides=(2,2)))
model.add(BatchNormalization())

model.add(Flatten())
#model.add(GlobalAveragePooling2D())

# 1 Dense
model.add(Dense(1024, activation='relu'))
#model.add(Flatten())
model.add(Dropout(0.3))


#2 Dense final layer
model.add(Dense(176, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 200, 100, 16)      448       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 100, 50, 16)       0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 100, 50, 16)       64        
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 100, 50, 32)       4640      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 50, 25, 32)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 50, 25, 32)        128       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 50, 25, 64)        18496     
__________

In [8]:
#Load weights from previous run
#model.load_weights(filepath='cnn_try1_2.h5')
### TODO: specify the number of epochs that you would like to use to train the model.

epochs = 20
batch_size = 30
learning_rate = 0.001
decay_rate = 1e-6
#decay_rate = learning_rate / epochs
momentum = 0.8
sgd = SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)
model.load_weights('cnn_ImageGenerator_2.ipynb.h5')
#model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
#print(train_tensors_1.shape)
#print(train_tensors_1.shape)

In [9]:
from keras.callbacks import ModelCheckpoint  



checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.from_scratch.hdf5', 
                               verbose=0, save_best_only=True)

train_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(
        trainingdata_spectrograms + '/' + 'train',
        target_size=(200, 100),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle = False)

test_datagen = ImageDataGenerator()
validation_generator = test_datagen.flow_from_directory(
        trainingdata_spectrograms + '/' + 'valid',
        target_size=(200, 100),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle = False)



Found 52625 images belonging to 176 classes.
Found 13551 images belonging to 176 classes.


In [10]:
model.fit_generator(
        train_generator,
        steps_per_epoch=52625 // batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=13551 // batch_size)


#model.fit(np.vstack([train_tensors_1, train_tensors_1]), train_targets, 
#          validation_split = 0.20,
#          epochs=epochs, batch_size=30, callbacks=[checkpointer], verbose=1)


model.save('cnn_ImageGenerator_2.ipynb.h5')

Epoch 1/20
1754/1754 [==============================] - 121s 69ms/step - loss: 0.0268 - acc: 0.9975 - val_loss: 3.1816 - val_acc: 0.2913
Epoch 2/20
1754/1754 [==============================] - 120s 68ms/step - loss: 0.0239 - acc: 0.9981 - val_loss: 3.2096 - val_acc: 0.2924
Epoch 3/20
1754/1754 [==============================] - 119s 68ms/step - loss: 0.0213 - acc: 0.9985 - val_loss: 3.1346 - val_acc: 0.3008
Epoch 4/20
1754/1754 [==============================] - 119s 68ms/step - loss: 0.0185 - acc: 0.9991 - val_loss: 2.9604 - val_acc: 0.3288
Epoch 5/20
1754/1754 [==============================] - 119s 68ms/step - loss: 0.0170 - acc: 0.9990 - val_loss: 2.8810 - val_acc: 0.3346
Epoch 6/20
1754/1754 [==============================] - 119s 68ms/step - loss: 0.0155 - acc: 0.9993 - val_loss: 2.9379 - val_acc: 0.3286
Epoch 7/20
1754/1754 [==============================] - 120s 68ms/step - loss: 0.0141 - acc: 0.9995 - val_loss: 3.0551 - val_acc: 0.3177
Epoch 8/20
1754/1754 [===================

In [22]:
#Create test tensors and check results.

testingdata_directory = '/home/ubuntu/src/capstone/datafiles/testingdata/'
testingdata_spectrograms = '/home/ubuntu/src/capstone/datafiles/testingdata/testing_spectrograms_complete'

#data = load_files(testingdata_spectrograms )
#print('type(data): ', type(data))
#test_files = np.array(data['filenames'])
#test_tensors =  paths_to_tensor(test_files).astype('float32')/255
#test_tensors2 =  paths_to_tensor(test_files[6159:]).astype('float32')/255
# print statistics about the dataset
#print('There are %s total test spectrgrams.\n' % len(test_tensors))


datagen = ImageDataGenerator(rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_generator = datagen.flow_from_directory(
        trainingdata_spectrograms + '/' + 'valid',
        target_size=(200, 100),
        batch_size=batch_size,
        class_mode='categorical',  # this means our generator will only yield batches of data, no labels
        shuffle=False)
# the predict_generator method returns the output of a model, given
# a generator that yields batches of numpy data
bottleneck_features_train = model.predict_generator(test_generator, verbose=1)
#bottleneck_features_train = [model.predict(np.expand_dims(test_tensor, axis=0), verbose=0) for test_tensor in test_tensors]
#test_datagen = ImageDataGenerator(rescale=1./255)
print('len(bottleneck_features_train1)', len(bottleneck_features_train))

Found 13551 images belonging to 176 classes.
452/452 [==============================] - 99s 219ms/step
len(bottleneck_features_train1) 13551


In [ ]:
#test_target_prediction = [np.argmax(model.predict(np.expand_dims(tensor, axis=0))) for tensor in test_tensors]
#print('len(test_target_prediction)', len(test_target_prediction))
#test_target_prediction = model.predict(test_tensors, )

In [24]:
print(len(bottleneck_features_train[100]))
print(np.argmax(bottleneck_features_train[0]))
language_ids = [np.argmax(language_prediction) for language_prediction in bottleneck_features_train]
#print(len(language_ids))
#Take list of test image files used by the generator during prediction.
test_gen_files = test_generator.filenames
#change file extensions to mp3.
test_files = []
for file in test_gen_files:
    filename, extension = os.path.splitext(file)
    filename = filename[5:] + '.mp3'
    test_files.append(filename)

class_index = pd.DataFrame(list(train_generator.class_indices.items()))

test_targets =[]
for id in language_ids:
    test_targets.append(class_index[0][id])

#create test_file, test_target dataFrame
test_output = pd.DataFrame(test_targets, test_files)
test_output.to_csv('test_output.csv')

176
0


In [4]:
import os
file = 'gprzb5txc5o.4096-128.png'

os.path.basename(file).split('.')

['gprzb5txc5o', '4096-128', 'png']